In [1]:
import os
import cv2

# INPUT folder containing s1 to s40 folders
input_root = 'archive_5'       # <- change this to your folder name (e.g., 'att_faces')
output_root = 'atnt_images'               # <- folder to save PNGs (will be created)

# Choose output image format
output_format = 'png'  # change to 'jpg' if you prefer JPEG

# Create output root directory if it doesn't exist
os.makedirs(output_root, exist_ok=True)

# Traverse subject folders
for subject in sorted(os.listdir(input_root)):
    subject_path = os.path.join(input_root, subject)
    
    if os.path.isdir(subject_path):
        output_subject_path = os.path.join(output_root, subject)
        os.makedirs(output_subject_path, exist_ok=True)

        # Traverse each .pgm file inside subject folder
        for file in sorted(os.listdir(subject_path)):
            if file.endswith('.pgm'):
                pgm_file_path = os.path.join(subject_path, file)
                image = cv2.imread(pgm_file_path, cv2.IMREAD_GRAYSCALE)

                # Create output filename
                filename_wo_ext = os.path.splitext(file)[0]
                output_file_path = os.path.join(output_subject_path, f"{filename_wo_ext}.{output_format}")

                # Save as PNG or JPG
                cv2.imwrite(output_file_path, image)

print(f"✅ Conversion complete. All .pgm images saved as .{output_format} in '{output_root}' folder.")


✅ Conversion complete. All .pgm images saved as .png in 'atnt_images' folder.


Starting AT&T face dataset processing...

Processing: s1

Processing: s10

Processing: s11

Processing: s12

Processing: s13

Processing: s14

Processing: s15

Processing: s16

Processing: s17

Processing: s18

Processing: s19

Processing: s2

Processing: s20

Processing: s21

Processing: s22

Processing: s23

Processing: s24

Processing: s25

Processing: s26

Processing: s27

Processing: s28

Processing: s29

Processing: s3

Processing: s30

Processing: s31

Processing: s32

Processing: s33
  No face found in 10.png
  No face found in 4.png
  No face found in 8.png

Processing: s34

Processing: s35
  No face found in 2.png

Processing: s36

Processing: s37

Processing: s38

Processing: s39

Processing: s4

Processing: s40

Processing: s5

Processing: s6

Processing: s7

Processing: s8

Processing: s9

✅ Facial feature data saved to: processed_faces_newFinal10\facial_features_and_landmarks_data.csv


In [4]:
import cv2
import dlib
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt # For Eigenface visualizations

# --- GLOBAL CONFIGURATIONS ---
# Load detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(
    "models/shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat"
)

# Define the root directories for input and output
root_input_dir = "atnt_images" # Changed to your new input directory
root_output_dir = "processed_faces_newFinal10"

# Define the scaling factor for upscaling images (for Dlib part)
scale_factor = 12

# List of subject folders (s1 to s40)
subject_folders = [f"s{i}" for i in range(1, 41)] # Dynamically create s1, s2, ..., s40

# Output directory for Eigenface related visualizations and data
eigenface_output_dir = os.path.join("processed_faces_newFinal10/eigenface_analysis_results")

# Ensure all output root directories exist
os.makedirs(root_output_dir, exist_ok=True)
os.makedirs(eigenface_output_dir, exist_ok=True) # Create this now


# --- Helper Function for Drawing Shapes ---
def draw_facial_shapes(image, landmarks):
    """
    Draws different geometric shapes on a given image based on facial landmarks.
    Args:
        image (numpy.ndarray): The image on which to draw.
        landmarks (dlib.full_object_detection): Dlib's landmark object.
    Returns:
        numpy.ndarray: The image with shapes drawn.
    """
    drawn_image = image.copy()

    color_eyebrows = (0, 255, 0)
    color_eyes = (255, 0, 0)
    color_nose_bridge = (0, 255, 255)
    color_nose_tip = (255, 255, 0)
    color_lips = (0, 165, 255)
    color_cheeks = (128, 0, 128)
    thickness = 2

    # Eyebrows (polylines)
    for i in range(17, 21):
        pt1 = (landmarks.part(i).x, landmarks.part(i).y)
        pt2 = (landmarks.part(i + 1).x, landmarks.part(i + 1).y)
        cv2.line(drawn_image, pt1, pt2, color_eyebrows, thickness)
    for i in range(22, 26):
        pt1 = (landmarks.part(i).x, landmarks.part(i).y)
        pt2 = (landmarks.part(i + 1).x, landmarks.part(i + 1).y)
        cv2.line(drawn_image, pt1, pt2, color_eyebrows, thickness)

    # Eyes (Ellipses)
    left_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)])
    if len(left_eye_points) >= 5:
        try:
            (center_le, axes_le, angle_le) = cv2.fitEllipse(left_eye_points)
            if axes_le[0] > 0 and axes_le[1] > 0:
                cv2.ellipse(drawn_image, (int(center_le[0]), int(center_le[1])),
                            (int(axes_le[0] / 2), int(axes_le[1] / 2)),
                            angle_le, 0, 360, color_eyes, thickness)
        except cv2.error: pass

    right_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)])
    if len(right_eye_points) >= 5:
        try:
            (center_re, axes_re, angle_re) = cv2.fitEllipse(right_eye_points)
            if axes_re[0] > 0 and axes_re[1] > 0:
                cv2.ellipse(drawn_image, (int(center_re[0]), int(center_re[1])),
                            (int(axes_re[0] / 2), int(axes_re[1] / 2)),
                            angle_re, 0, 360, color_eyes, thickness)
        except cv2.error: pass

    # Nose Bridge (Trapezium)
    nose_bridge_pts = np.array([
        (landmarks.part(27).x, landmarks.part(27).y), (landmarks.part(28).x, landmarks.part(28).y),
        (landmarks.part(29).x, landmarks.part(29).y), (landmarks.part(30).x, landmarks.part(30).y)
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [nose_bridge_pts], True, color_nose_bridge, thickness)

    # Nose Tip (Triangle)
    nose_tip_pts = np.array([
        (landmarks.part(30).x, landmarks.part(30).y), (landmarks.part(31).x, landmarks.part(31).y),
        (landmarks.part(35).x, landmarks.part(35).y)
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [nose_tip_pts], True, color_nose_tip, thickness)

    # Lips (Outer Lip Ellipse)
    outer_lip_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 60)])
    if len(outer_lip_points) >= 5:
        try:
            (center_ol, axes_ol, angle_ol) = cv2.fitEllipse(outer_lip_points)
            if axes_ol[0] > 0 and axes_ol[1] > 0:
                cv2.ellipse(drawn_image, (int(center_ol[0]), int(center_ol[1])),
                            (int(axes_ol[0] / 2), int(axes_ol[1] / 2)),
                            angle_ol, 0, 360, color_lips, thickness)
        except cv2.error: pass

    # Cheeks (Broader Triangles)
    left_cheek_pts = np.array([
        (landmarks.part(36).x, landmarks.part(36).y), (landmarks.part(2).x, landmarks.part(2).y),
        (landmarks.part(48).x, landmarks.part(48).y)
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [left_cheek_pts], True, color_cheeks, thickness)

    right_cheek_pts = np.array([
        (landmarks.part(45).x, landmarks.part(45).y), (landmarks.part(14).x, landmarks.part(14).y),
        (landmarks.part(54).x, landmarks.part(54).y)
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [right_cheek_pts], True, color_cheeks, thickness)

    return drawn_image

# --- Helper Functions for Curve Fitting ---

def fit_parabola_params(points):
    if len(points) < 3: return np.nan, np.nan, np.nan
    x_coords = np.array([p[0] for p in points])
    y_coords = np.array([p[1] for p in points])
    try:
        with np.errstate(all='raise'):
            coefficients = np.polyfit(x_coords, y_coords, 2)
            return tuple(coefficients)
    except Exception: return np.nan, np.nan, np.nan

def fit_ellipse_params(points):
    if len(points) < 5: return np.nan, np.nan, np.nan, np.nan, np.nan
    points_np = np.array(points, dtype=np.int32)
    try:
        (center, axes, angle) = cv2.fitEllipse(points_np)
        if axes[0] <= 0 or axes[1] <= 0: return np.nan, np.nan, np.nan, np.nan, np.nan
        return center[0], center[1], axes[0], axes[1], angle
    except cv2.error: return np.nan, np.nan, np.nan, np.nan, np.nan
    except Exception: return np.nan, np.nan, np.nan, np.nan, np.nan


# --- Main Processing Loop for Dlib, Drawing, and Initial Feature Extraction ---
print("Starting facial landmark detection, shape drawing, and feature extraction (Stage 1)...")

all_facial_features_data = []

for subject_id in subject_folders: # Loop through s1, s2, ..., s40
    input_subject_path = os.path.join(root_input_dir, subject_id)

    # Output paths adjusted to the new structure (subject_id instead of emotion)
    output_with_photo_path = os.path.join(root_output_dir, subject_id, f"{subject_id}_with_photo")
    output_without_photo_path = os.path.join(root_output_dir, subject_id, f"{subject_id}_without_photo")
    output_shapes_drawn_path = os.path.join(root_output_dir, subject_id, f"{subject_id}_shapes_drawn")

    os.makedirs(output_with_photo_path, exist_ok=True)
    os.makedirs(output_without_photo_path, exist_ok=True)
    os.makedirs(output_shapes_drawn_path, exist_ok=True)

    print(f"\nProcessing subject: {subject_id}...")

    for filename in os.listdir(input_subject_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            image_path = os.path.join(input_subject_path, filename)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Warning: Could not read image {image_path}. Skipping.")
                continue

            # print(f"  Processing file: {filename}") # Uncomment for more verbose output

            upscaled_image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
            gray = cv2.cvtColor(upscaled_image, cv2.COLOR_BGR2GRAY)

            faces = detector(gray)

            if len(faces) > 0:
                face = faces[0]
                landmarks = predictor(gray, face)

                # Store 'subject_id' instead of 'emotion'
                current_image_features = {'subject_id': subject_id, 'filename': filename}

                # Collect raw landmark data
                for n in range(68):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y
                    current_image_features[f'landmark_{n}_x'] = x
                    current_image_features[f'landmark_{n}_y'] = y

                # Apply Curve Fitting Algorithms and Store Parameters
                leb_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(17, 22)]
                a_leb, b_leb, c_leb = fit_parabola_params(leb_points)
                current_image_features['leb_parabola_a'] = a_leb
                current_image_features['leb_parabola_b'] = b_leb
                current_image_features['leb_parabola_c'] = c_leb

                reb_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(22, 27)]
                a_reb, b_reb, c_reb = fit_parabola_params(reb_points)
                current_image_features['reb_parabola_a'] = a_reb
                current_image_features['reb_parabola_b'] = b_reb
                current_image_features['reb_parabola_c'] = c_reb

                le_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)]
                cx_le, cy_le, maj_le, min_le, ang_le = fit_ellipse_params(le_points)
                current_image_features['le_ellipse_center_x'] = cx_le; current_image_features['le_ellipse_center_y'] = cy_le
                current_image_features['le_ellipse_major_axis'] = maj_le; current_image_features['le_ellipse_minor_axis'] = min_le
                current_image_features['le_ellipse_angle'] = ang_le

                re_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)]
                cx_re, cy_re, maj_re, min_re, ang_re = fit_ellipse_params(re_points)
                current_image_features['re_ellipse_center_x'] = cx_re; current_image_features['re_ellipse_center_y'] = cy_re
                current_image_features['re_ellipse_major_axis'] = maj_re; current_image_features['re_ellipse_minor_axis'] = min_re
                current_image_features['re_ellipse_angle'] = ang_re

                ol_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 60)]
                cx_ol, cy_ol, maj_ol, min_ol, ang_ol = fit_ellipse_params(ol_points)
                current_image_features['ol_ellipse_center_x'] = cx_ol; current_image_features['ol_ellipse_center_y'] = cy_ol
                current_image_features['ol_ellipse_major_axis'] = maj_ol; current_image_features['ol_ellipse_minor_axis'] = min_ol
                current_image_features['ol_ellipse_angle'] = ang_ol

                # Nose Bridge (Trapezium - extract vertices)
                current_image_features['nose_bridge_v1_x'] = landmarks.part(27).x; current_image_features['nose_bridge_v1_y'] = landmarks.part(27).y
                current_image_features['nose_bridge_v2_x'] = landmarks.part(28).x; current_image_features['nose_bridge_v2_y'] = landmarks.part(28).y
                current_image_features['nose_bridge_v3_x'] = landmarks.part(29).x; current_image_features['nose_bridge_v3_y'] = landmarks.part(29).y
                current_image_features['nose_bridge_v4_x'] = landmarks.part(30).x; current_image_features['nose_bridge_v4_y'] = landmarks.part(30).y

                # Nose Tip (Triangle - extract vertices)
                current_image_features['nose_tip_v1_x'] = landmarks.part(30).x; current_image_features['nose_tip_v1_y'] = landmarks.part(30).y
                current_image_features['nose_tip_v2_x'] = landmarks.part(31).x; current_image_features['nose_tip_v2_y'] = landmarks.part(31).y
                current_image_features['nose_tip_v3_x'] = landmarks.part(35).x; current_image_features['nose_tip_v3_y'] = landmarks.part(35).y

                # Left Cheek (Triangle - extract vertices)
                current_image_features['left_cheek_v1_x'] = landmarks.part(36).x; current_image_features['left_cheek_v1_y'] = landmarks.part(36).y
                current_image_features['left_cheek_v2_x'] = landmarks.part(2).x; current_image_features['left_cheek_v2_y'] = landmarks.part(2).y
                current_image_features['left_cheek_v3_x'] = landmarks.part(48).x; current_image_features['left_cheek_v3_y'] = landmarks.part(48).y

                # Right Cheek (Triangle - extract vertices)
                current_image_features['right_cheek_v1_x'] = landmarks.part(45).x; current_image_features['right_cheek_v1_y'] = landmarks.part(45).y
                current_image_features['right_cheek_v2_x'] = landmarks.part(14).x; current_image_features['right_cheek_v2_y'] = landmarks.part(14).y
                current_image_features['right_cheek_v3_x'] = landmarks.part(54).x; current_image_features['right_cheek_v3_y'] = landmarks.part(54).y

                all_facial_features_data.append(current_image_features)

                # Visualizations
                drawn_image_with_photo = upscaled_image.copy()
                drawn_image_without_photo = np.zeros(upscaled_image.shape, dtype=np.uint8)
                drawn_image_shapes_only = np.zeros(upscaled_image.shape, dtype=np.uint8)

                # Draw quadrant lines
                nose_x = landmarks.part(30).x; nose_y = landmarks.part(30).y
                img_h, img_w = drawn_image_with_photo.shape[:2]
                cv2.line(drawn_image_with_photo, (nose_x, 0), (nose_x, img_h), (0, 255, 0), 1)
                cv2.line(drawn_image_with_photo, (0, nose_y), (img_w, nose_y), (0, 255, 0), 1)

                # Draw landmarks and labels
                drawn_index = 1; seen = set()
                for n in range(68):
                    x = landmarks.part(n).x; y = landmarks.part(n).y
                    if (x, y) in seen: continue
                    seen.add((x, y))
                    circle_radius = 2; fill_type = -1
                    offset_x, offset_y = -6, 10
                    if n in [37, 38, 39, 43, 44, 45]: offset_y = -10
                    elif n == 49: offset_x = -20; offset_y = 6
                    elif n == 50: offset_x = 0; offset_y = -8
                    elif n == 65: offset_x = -14; offset_y = 10
                    elif 48 <= n <= 54: offset_y = -8
                    elif 55 <= n <= 59: offset_y = 10
                    elif 60 <= n <= 64: offset_y = -8
                    elif 65 <= n <= 67: offset_y = 10

                    cv2.circle(drawn_image_with_photo, (x, y), circle_radius, (0, 0, 255), fill_type)
                    cv2.putText(drawn_image_with_photo, str(drawn_index), (x + offset_x, y + offset_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 0), 1, cv2.LINE_AA)
                    cv2.circle(drawn_image_without_photo, (x, y), circle_radius, (255, 255, 255), fill_type)
                    cv2.putText(drawn_image_without_photo, str(drawn_index), (x + offset_x, y + offset_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (255, 255, 255), 1, cv2.LINE_AA)
                    drawn_index += 1

                drawn_image_shapes_only = draw_facial_shapes(drawn_image_shapes_only, landmarks)

                output_image_path_with_photo = os.path.join(output_with_photo_path, filename)
                cv2.imwrite(output_image_path_with_photo, drawn_image_with_photo)
                output_image_path_without_photo = os.path.join(output_without_photo_path, filename)
                cv2.imwrite(output_image_path_without_photo, drawn_image_without_photo)
                output_image_path_shapes_drawn = os.path.join(output_shapes_drawn_path, filename)
                cv2.imwrite(output_image_path_shapes_drawn, drawn_image_shapes_only)

            else:
                print(f"  No face detected in {filename} from {subject_id} folder.")

print("\nStage 1: Dlib processing, shape drawing, and initial feature extraction complete!")

# Save all collected facial feature data to a single CSV file
if all_facial_features_data:
    df = pd.DataFrame(all_facial_features_data)
    df = df.reindex(columns=sorted(df.columns))
    csv_output_path = os.path.join(root_output_dir, "facial_features_and_landmarks_data.csv")
    df.to_csv(csv_output_path, index=False)
    print(f"All facial landmark coordinates and fitted shape parameters saved to: {csv_output_path}")
else:
    print("No facial feature data was collected in Stage 1.")


# --- Configuration for Eigenface Decomposition ---
# Define the target size for all face images (Width, Height)
face_target_size = (100, 100)

# Number of top Eigenfaces to retain
num_eigenfaces_to_keep = 50

# --- Step 1: Data Preparation for Eigenfaces ---
print("\nStarting Stage 2: Eigenface Decomposition...")
print("Step 1: Collecting and preprocessing face images for PCA...")

all_face_vectors = []
image_metadata_pca = [] # Use a separate list for PCA metadata to avoid conflicts

for subject_id in subject_folders: # Loop through s1, s2, ..., s40
    # Read from the '_shapes_drawn' folder, as these are upscaled and have shapes,
    # but more importantly, they are already resized consistently by the previous step's upscaling.
    # For purest Eigenfaces, ideally these would be tight crops of the face only.
    subject_dir = os.path.join(root_output_dir, subject_id, f"{subject_id}_shapes_drawn") # Adjusted path
    if not os.path.exists(subject_dir):
        print(f"Warning: Directory '{subject_dir}' not found. Skipping {subject_id} for PCA.")
        continue

    for filename in os.listdir(subject_dir):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            image_path = os.path.join(subject_dir, filename)
            img = cv2.imread(image_path)

            if img is None:
                print(f"Warning: Could not read image {image_path}. Skipping for PCA.")
                continue

            # Convert to grayscale
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Resize to uniform size (ensure consistency, even if already done by upscaling in stage 1)
            # This is important as Dlib's upscaling is a factor, not a fixed size.
            resized_face = cv2.resize(gray_img, face_target_size, interpolation=cv2.INTER_AREA)

            # Flatten the 2D image into a 1D vector
            flattened_face = resized_face.flatten()

            all_face_vectors.append(flattened_face)
            image_metadata_pca.append({'subject_id': subject_id, 'filename': filename}) # Store subject_id

if not all_face_vectors:
    print("No face images found or processed for Eigenface decomposition. Exiting.")
    # You might want to exit the entire notebook or just skip this section
    # For a notebook, print and return might be more appropriate.
    # return
else: # Only proceed if there's data
    X = np.array(all_face_vectors, dtype=np.float64)
    num_images, num_pixels = X.shape
    print(f"Collected {num_images} images, each with {num_pixels} pixels for PCA.")

    # --- Step 2: Calculate the Mean Face ---
    print("Step 2: Calculating the mean face...")
    mean_face_vector = np.mean(X, axis=0)
    mean_face_image = mean_face_vector.reshape(face_target_size)

    plt.figure(figsize=(4, 4))
    plt.imshow(mean_face_image, cmap='gray')
    plt.title("Mean Face")
    plt.axis('off')
    mean_face_path = os.path.join(eigenface_output_dir, "mean_face.png")
    plt.savefig(mean_face_path)
    plt.close()
    print(f"Mean face saved to: {mean_face_path}")

    # --- Step 3: Subtract the Mean Face ---
    print("Step 3: Centering the data (subtracting mean face)...")
    X_centered = X - mean_face_vector

    # --- Step 4 & 5: Compute Eigenvalues and Eigenvectors ---
    print("Step 4 & 5: Computing eigenvectors and eigenvalues...")
    S_matrix = X_centered @ X_centered.T # N x N scatter matrix
    eigenvalues_small, eigenvectors_small = np.linalg.eigh(S_matrix)

    sorted_indices = np.argsort(eigenvalues_small)[::-1]
    eigenvalues_small = eigenvalues_small[sorted_indices]
    eigenvectors_small = eigenvectors_small[:, sorted_indices]

    eigenfaces_raw = X_centered.T @ eigenvectors_small
    eigenfaces = eigenfaces_raw / np.linalg.norm(eigenfaces_raw, axis=0) # Normalize

    print("Eigenvalues and eigenvectors computed.")

    # --- Step 6: Select Principal Components (Eigenfaces) ---
    print(f"Step 6: Selecting top {num_eigenfaces_to_keep} Eigenfaces...")
    selected_eigenfaces = eigenfaces[:, :num_eigenfaces_to_keep]
    selected_eigenvalues = eigenvalues_small[:num_eigenfaces_to_keep]

    print("Visualizing top Eigenfaces...")
    fig, axes = plt.subplots(1, min(num_eigenfaces_to_keep, 10), figsize=(20, 4))
    for i, ax in enumerate(axes):
        eigenface_img = selected_eigenfaces[:, i].reshape(face_target_size)
        ax.imshow(eigenface_img, cmap='gray')
        ax.set_title(f'Eigenface {i+1}')
        ax.axis('off')
    plt.tight_layout()
    eigenfaces_plot_path = os.path.join(eigenface_output_dir, "top_eigenfaces.png")
    plt.savefig(eigenfaces_plot_path)
    plt.close()
    print(f"Top Eigenfaces visualization saved to: {eigenfaces_plot_path}")

    # --- Step 7: Project Faces onto Eigenface Space ---
    print("Step 7: Projecting original faces onto Eigenface space...")
    feature_vectors = X_centered @ selected_eigenfaces
    print(f"Projected {num_images} faces into a {num_eigenfaces_to_keep}-dimensional Eigenface space.")

    # --- Step 8: Store Eigenfaces and Feature Vectors ---
    print("Step 8: Storing feature vectors in CSV...")
    df_features = pd.DataFrame(feature_vectors)
    df_features.columns = [f'eigen_feature_{i}' for i in range(num_eigenfaces_to_keep)]
    df_metadata = pd.DataFrame(image_metadata_pca) # Use the correct metadata list for PCA
    final_df_pca = pd.concat([df_metadata, df_features], axis=1)

    features_csv_path_pca = os.path.join(eigenface_output_dir, "eigenface_features.csv")
    final_df_pca.to_csv(features_csv_path_pca, index=False)

    print(f"Eigenface feature vectors saved to: {features_csv_path_pca}")
    print("Stage 2: Eigenface decomposition complete!")

Starting facial landmark detection, shape drawing, and feature extraction (Stage 1)...

Processing subject: s1...

Processing subject: s2...

Processing subject: s3...

Processing subject: s4...

Processing subject: s5...

Processing subject: s6...

Processing subject: s7...

Processing subject: s8...

Processing subject: s9...

Processing subject: s10...

Processing subject: s11...

Processing subject: s12...

Processing subject: s13...

Processing subject: s14...

Processing subject: s15...

Processing subject: s16...

Processing subject: s17...

Processing subject: s18...

Processing subject: s19...

Processing subject: s20...

Processing subject: s21...

Processing subject: s22...

Processing subject: s23...

Processing subject: s24...

Processing subject: s25...

Processing subject: s26...

Processing subject: s27...

Processing subject: s28...

Processing subject: s29...

Processing subject: s30...

Processing subject: s31...

Processing subject: s32...

Processing subject: s33..